In [1]:
# Importing packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from tqdm import tqdm
import logging
from preprocessing_functions import *

# Load one month dataset to test  new version preprocess

In [2]:
# Set up logging, mainly to get timings easily.
logging.basicConfig(
    format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.DEBUG)

################ Relative Humidity Conversion Functions #########################
def eliq(T):
    a_liq = np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]);
    c_liq = -80.0
    T0 = 273.16
    return 100.0*np.polyval(a_liq,np.maximum(c_liq,T-T0))

def eice(T):
    a_ice = np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]);
    c_ice = np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07])
    T0 = 273.16
    return np.where(T>c_ice[0],eliq(T),\
                   np.where(T<=c_ice[1],100.0*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),100.0*np.polyval(a_ice,T-T0)))

def esat(T):
    T0 = 273.16
    T00 = 253.16
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(0.0,np.minimum(1.0,omtmp))
    return np.where(T>T0,eliq(T),np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

def RH(T,qv,P0,PS,hyam,hybm):
    R = 287.0
    Rv = 461.0
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    return Rv*p*qv/(R*esat(T))

########### Load Data funcitons #########################################
def ls(data_path = ""):
    return os.popen(" ".join(["ls", data_path])).read().splitlines()

def load_data(month, year, data_path):
    datasets = ls(data_path)
    if month:
        month = str(month).zfill(2) # For several month datasets, month belongs to [1, 12]
        year = str(year).zfill(4)
        datasets = [data_path + x for x in datasets if "h1." + year + "-" + month in x]
    else:
        year = str(year).zfill(4) # For whole year datasets, month = 0
        datasets = [data_path + x for x in datasets if "h1." + year  in x]
    datasets = xr.open_mfdataset(datasets, decode_times=True, chunks={'time': 960, 'latitude':10, 'longitude':10}, parallel=True)
    return datasets


# ds = ! ls {data_path} | grep 'h1.0000-*'
# ds = [data_path + i for i in ds]
# %time ds = xr.open_mfdataset(ds, decode_times=True, chunks={'time': 960, 'latitude':1, 'longitude':1}, parallel=True)

#########################################################################

def var_time_select_ds(ds, input_var):
    tmp1 = ds[input_var[:4]].isel(time=slice(1, None)) # not the first time
    tmp2 = ds[input_var[4:]].isel(time=slice(None, -1)) # not the last time
    tmp2['time'] =tmp1['time'] # change the time coordinate for later concatenate
    nn_input = xr.merge([tmp1, tmp2])
    return nn_input

def stack_ds(ds):
    
    var_list = list(ds.data_vars)
    multi_layer_vars = [var for var in var_list if 'lev' in ds[var].coords]
    
    for var in multi_layer_vars:
        for i in range(ds.lev.size):
            ds[var+str(i)] = ds[var].isel(lev=i)
            
    ds = ds.drop(multi_layer_vars)
    ds = ds.drop_dims('lev')
    ds = ds.stack(sample=('time', 'lat', 'lon'))
    ds = ds.reset_index('sample')
    return ds

def make_nn_input_target(spData, family='relative', subsample = False, spacing = 8, contiguous=False, print_diagnostics=False):
    
    if subsample:
        logging.info('Subsample dataset by longitude with spacing = {}'.format(spacing))
        spData = spData.isel(lon=slice(0, None, spacing))
        
    ##### Calculate humidity #########################################
    R = 287.0
    Rv = 461.0
    p = spData['P0'] * spData['hyam'] + spData['NNPS'] * spData['hybm']
    spData['NEWHUM'] = Rv*p*spData['NNQBP'] / (R*esat(spData['NNTBP']))
    
    if family == 'specific':
        input_var = ['NNTBP', 'NNQBP', 'NNPS', 'SOLIN', 'SHFLX', 'LHFLX']
    elif family == 'relative':
        input_var = ['NNTBP', 'NEWHUM', 'NNPS', 'SOLIN', 'SHFLX', 'LHFLX']
        
    nn_input = var_time_select_ds(spData, input_var)

    if print_diagnostics:
        print("nnInput",nn_input.dims)
        newhum = nn_input['NEWHUM']
        oldhum = spData["RELHUM"].isel(time=slice(1, None, 1))
        errors = (newhum-oldhum/100).values.flatten()
        result = "Mean relative humidity conversion error: " + str(np.mean(errors)) + "\n"
        result = result + "Variance for relative humidity conversion error: " + str(np.var(errors)) + "\n"
        print(result)
        
    #########  make nn_target_ds ####################################
    target_var = ['TPHYSTND', 'PHQ']
    nn_target = spData[target_var].isel(time=slice(1, None, 1))
    
    ######### Contiguous ############################################
    if not contiguous:
            nn_input = nn_input.isel(time=slice(None, -1,1)) #the last timestep of a run can have funky values
            nn_target = nn_target.isel(time=slice(None, -1, 1))
    
    return nn_input, nn_target     

def combine():
    """
    This method could be replaced in ditributed way, using more than one cpu to process these datasets in parallel.
    Dask.distribute or xarray-spark will be the one to figure out the problem.
    """
    pass

def save_file(path, ds):
    ds.to_netcdf(path)
    
def preprocess(month, data_path='/ocean/projects/atm200007p/jlin96/longSPrun/'):
    
    logging.info('Reading input files')
    spData = load_data(month=month, year = 0, data_path = data_path)
    
    logging.info('Calculate relative humidity and generate unstacked input and target datasets')
    nn_input, nn_target = make_nn_input_target(spData, family='relative', subsample=True)
    
    logging.info('Stack and reshape input and target datasets')
    stacked_nn_input = stack_ds(nn_input)
    stacked_nn_target = stack_ds(nn_target)

    
#     if save_file:
#         logging.info('Save input and target datasets')
#         stacked_nn_input.to_netcdf('stacked_nn_input.nc')
#         stacked_nn_target.to_netcdf('stacked_nn_target.nc')

    logging.info('Preprocess done!')
    return stacked_nn_input, stacked_nn_target
  

In [16]:
if __name__ == '__main__':
    preprocess(month=2)

12/21/2022 06:39:25 PM Reading input files
12/21/2022 06:39:26 PM Calculate relative humidity and generate unstacked input and target datasets
12/21/2022 06:39:26 PM Subsample dataset by longitude with spacing = 8
12/21/2022 06:40:41 PM Stack and reshape input and target datasets


select var and time


12/21/2022 06:40:41 PM Preprocess done!


# Test dask.distribute for calculating one month dataset in parallel

In [3]:
from distributed import Client

In [4]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='5GB')

12/22/2022 12:21:37 PM Using selector: EpollSelector


In [25]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 9.31 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46535,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: 13 minutes ago,Total memory: 9.31 GiB
Comm: tcp://127.0.0.1:37417,Total threads: 2
Dashboard: http://127.0.0.1:35445/status,Memory: 4.66 GiB
Nanny: tcp://127.0.0.1:38083,


In [22]:
future = client.submit(preprocess)

In [23]:
stacked_nn_input = future.result()[0]
stacked_nn_input

<xarray.Dataset>
Dimensions:   (sample: 1374208)
Coordinates:
    time      (sample) object 0000-02-01 00:30:00 ... 0000-02-28 23:00:00
    lat       (sample) float64 -87.86 -87.86 -87.86 -87.86 ... 87.86 87.86 87.86
    lon       (sample) float64 0.0 22.5 45.0 67.5 ... 270.0 292.5 315.0 337.5
Dimensions without coordinates: sample
Data variables: (12/64)
    NNPS      (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    SOLIN     (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    SHFLX     (sample) float32 dask.array<chunksize=(49152,), meta=np.ndarray>
    LHFLX     (sample) float32 dask.array<chunksize=(49152,), meta=np.ndarray>
    NNTBP0    (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    NNTBP1    (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    ...        ...
    NEWHUM24  (sample) float64 dask.array<chunksize=(48128,), meta=np.ndarray>
    NEWHUM25  (sample) float64 dask.array<chunksize=(48128,), meta=np.ndarray>
    NEWHUM26  (sample) float64 dask.array<chunksize=(48128,), meta=np.ndarray>
    NEWHUM27  (sample) float64 dask.array<chunksize=(48128,), meta=np.ndarray>
    NEWHUM28  (sample) float64 dask.array<chunksize=(48128,), meta=np.ndarray>
    NEWHUM29  (sample) float64 dask.array<chunksize=(48128,), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    source:       CAM
    case:         AndKua_aqua_Base_training
    title:        
    logname:      jlin96
    host:         
    Version:      $Name:  $
    revision_Id:  $Id: history.F90,v 1.26.2.38 2003/12/15 18:52:35 hender Exp $

In [24]:
stacked_nn_target = future.result()[1]
stacked_nn_target

<xarray.Dataset>
Dimensions:     (sample: 1374208)
Coordinates:
    time        (sample) object 0000-02-01 00:30:00 ... 0000-02-28 23:00:00
    lat         (sample) float64 -87.86 -87.86 -87.86 ... 87.86 87.86 87.86
    lon         (sample) float64 0.0 22.5 45.0 67.5 ... 270.0 292.5 315.0 337.5
Dimensions without coordinates: sample
Data variables: (12/60)
    TPHYSTND0   (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    TPHYSTND1   (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    TPHYSTND2   (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    TPHYSTND3   (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    TPHYSTND4   (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    TPHYSTND5   (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    ...          ...
    PHQ24       (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    PHQ25       (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    PHQ26       (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    PHQ27       (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    PHQ28       (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
    PHQ29       (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    source:       CAM
    case:         AndKua_aqua_Base_training
    title:        
    logname:      jlin96
    host:         
    Version:      $Name:  $
    revision_Id:  $Id: history.F90,v 1.26.2.38 2003/12/15 18:52:35 hender Exp $

# Test dask.distribute for one year datasets 

In [1]:
! lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              128
On-line CPU(s) list: 0-127
Thread(s) per core:  1
Core(s) per socket:  64
Socket(s):           2
NUMA node(s):        2
Vendor ID:           AuthenticAMD
CPU family:          23
Model:               49
Model name:          AMD EPYC 7742 64-Core Processor
Stepping:            0
CPU MHz:             3297.424
BogoMIPS:            4491.29
Virtualization:      AMD-V
L1d cache:           32K
L1i cache:           32K
L2 cache:            512K
L3 cache:            16384K
NUMA node0 CPU(s):   0-63
NUMA node1 CPU(s):   64-127
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid aperfmperf pni pclmulqdq monitor ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand lahf_lm cmp_legacy svm extapi

In [2]:
! free -h

free: /jet/home/yxia1/.conda/envs/practice-env/lib/libuuid.so.1: no version information available (required by /lib64/libblkid.so.1)
              total        used        free      shared  buff/cache   available
Mem:          251Gi       7.5Gi       214Gi        14Mi        29Gi       242Gi
Swap:         699Gi       7.0Mi       699Gi


## For one month datasets, time consuming 1min34s

In [3]:
from distributed import Client, progress
client = Client(n_workers=4)
client

12/22/2022 01:25:42 PM Using selector: EpollSelector


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41601,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:41377,Total threads: 1
Dashboard: http://127.0.0.1:33181/status,Memory: 62.50 GiB
Nanny: tcp://127.0.0.1:45137,


In [4]:
# Test for one month datasets
future = client.submit(preprocess, month=2)
progress(future) # time consuming: 1min34s

VBox()

# For one year datasets, program always shuts down beacuse of memory

In [3]:
# Test for one year data
from distributed import Client, progress
client = Client(n_workers=12)
future = client.submit(preprocess, month=0)
progress(future)

12/22/2022 01:33:24 PM Using selector: EpollSelector


VBox()

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 14.62 GiB -- Worker memory limit: 20.83 GiB


## Instead to load one month data one by one and then concatenate 

In [ ]:
from distributed import Client, progress
from time import sleep
inputs, targets = [], []
for i in range(1,13):
    client = Client(n_workers=4)
    future = client.submit(preprocess, month=i)
#     inputs = xr.concat(future.result()[0])
    inputs.append(future.result()[0])
    targets.append(future.result()[1])
    client.close()
    sleep(2)


12/22/2022 02:16:02 PM Using selector: EpollSelector


In [4]:
inputs

[<xarray.Dataset>
 Dimensions:   (sample: 1521664)
 Coordinates:
     time      (sample) object 0000-01-01 00:30:00 ... 0000-01-31 23:00:00
     lat       (sample) float64 -87.86 -87.86 -87.86 -87.86 ... 87.86 87.86 87.86
     lon       (sample) float64 0.0 22.5 45.0 67.5 ... 270.0 292.5 315.0 337.5
 Dimensions without coordinates: sample
 Data variables: (12/64)
     NNPS      (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
     SOLIN     (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
     SHFLX     (sample) float32 dask.array<chunksize=(49152,), meta=np.ndarray>
     LHFLX     (sample) float32 dask.array<chunksize=(49152,), meta=np.ndarray>
     NNTBP0    (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
     NNTBP1    (sample) float32 dask.array<chunksize=(48128,), meta=np.ndarray>
     ...        ...
     NEWHUM24  (sample) float64 dask.array<chunksize=(48128,), meta=np.ndarray>
     NEWHUM25  (sample) float64 dask.array<chunksize=(

In [ ]:
one_year_nn_inputs = xr.concat(inputs, dim='sample')
one_year_nn_targets = xr.concat(targets, dim='sample')

In [16]:
#  Fast way to save data time ncdat.load().to_netcdf('test_faster.nc')
one_year_nn_inputs.to_netcdf('../training/training_data/one_year_nn_inputs_spacing_8.nc')
one_year_nn_targets.to_netcdf('../training/training_data/one_year_nn_targets_spacing_8.nc')